In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import time
import sys
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

np.set_printoptions(threshold=sys.maxsize)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from keras.layers import AvgPool2D, BatchNormalization, Reshape, Activation
from keras.optimizers import Adadelta, RMSprop, Adam
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from keras.layers import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf



In [ ]:
folder="/content/drive/My Drive/Colab Notebooks/PQ19/data/"

In [ ]:
df = pd.read_csv(folder+'df191.csv')
df = shuffle(df)
df.head()

In [ ]:
dataset=df.values

In [ ]:
X_train=dataset[:26600,1:2001]
Y_train=dataset[:26600,0]
X_val=dataset[26600:30400,1:2001]
Y_val=dataset[26600:30400,0]
X_test=dataset[30400:38000,1:2001]
Y_test=dataset[30400:38000,0]
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape )

In [ ]:
import collections, numpy
collections.Counter(Y_val)

DCNN

In [ ]:
img_rows = 40
img_cols = 50

X_train1 = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_val1 = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
X_test1 = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
print(X_train1.shape, X_val1.shape, X_test1.shape, Y_train.shape, Y_val.shape, Y_test.shape )

input_shape = (img_rows, img_cols, 1)

In [ ]:
activation='relu'

def create_model():
  model = Sequential()   
  model.add(Conv2D(128, kernel_size=(3, 3), activation=activation, input_shape=input_shape)) 
  model.add(MaxPooling2D(2,2))

  model.add(Conv2D(64, (3, 3), activation=activation))
  model.add(MaxPooling2D(2,2))

  model.add(Conv2D(64, kernel_size=(3, 3), activation=activation, input_shape=input_shape)) 
  model.add(MaxPooling2D(2,2))

  model.add(Flatten())

  model.add(Dense(256, activation=activation))
  model.add(BatchNormalization())
  model.add(Dense(19, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

  return model



In [ ]:
from keras.callbacks import ModelCheckpoint

model_ckpt= create_model()
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/PQ19/data/cp191.ckpt"
cp_callback = ModelCheckpoint(filepath=checkpoint_path,save_best_only=True, save_weights_only=True, verbose=1)

model_ckpt.summary()

In [ ]:
batchsize = 100
epochs = 1000

hist=model_ckpt.fit(X_train1, Y_train, batch_size=batchsize, epochs=epochs, validation_data=(X_val1, Y_val),callbacks=[cp_callback], verbose=1)




In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')  
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
#get the latest checkpoint file

model_latest_checkpoint = create_model()
# Load the previously saved weights
model_latest_checkpoint.load_weights(checkpoint_path)

In [ ]:

acc = model_latest_checkpoint.evaluate(X_test1, Y_test)
print("Loss:", acc[0], " Accuracy:", acc[1])

pred = model_latest_checkpoint.predict(X_test1)
y_pred = pred.argmax(axis=-1)
y_true = Y_test

In [ ]:
output_classes=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19']

def plot_confusion_matrix(y_true, y_pred, classifier_name):
    mtx = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(20,20))
    ax.set_title(classifier_name)
    sns.heatmap(mtx, xticklabels= output_classes, yticklabels=output_classes, cmap="Blues", annot=True, fmt='d', linewidths=.5,  cbar=False, ax=ax)
    #  square=True,
    plt.ylabel('true label')
    plt.xlabel('predicted label')

In [ ]:
plot_confusion_matrix(y_true, y_pred, classifier_name='DCNN')
print(classification_report(y_true, y_pred,digits=4))
acc_cnn1 = accuracy_score(y_true, y_pred)
print(acc_cnn1)

DLSTM

In [ ]:
img_rows = 40
img_cols = 50

X_train1 = X_train.reshape(X_train.shape[0], img_rows, img_cols)
X_val1 = X_val.reshape(X_val.shape[0], img_rows, img_cols)
X_test1 = X_test.reshape(X_test.shape[0], img_rows, img_cols)
print(X_train1.shape, X_val1.shape, X_test1.shape, Y_train.shape, Y_val.shape, Y_test.shape )

input_shape = (img_rows, img_cols)

In [ ]:
activation='relu'

def lstm_model_1(activation):
    
    model = tf.keras.models.Sequential()
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(100, input_shape=input_shape, return_sequences=True))
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(100, return_sequences=True))
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(100))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(19, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

model_lstm_1 = lstm_model_1(activation)
model_lstm_1.summary()

In [ ]:
batchsize = 100
epochs = 100

histlstm=model_lstm_1.fit(X_train1, Y_train, batch_size=batchsize, epochs=epochs, validation_data=(X_val1, Y_val), verbose=1)

with open('/content/drive/My Drive/Colab Notebooks/PQ19/data/lstm1.pkl', 'wb') as file:  
    pickle.dump(model, file)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(histlstm.history['loss'])
plt.plot(histlstm.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

plt.plot(histlstm.history['accuracy'])
plt.plot(histlstm.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
#get the latest checkpoint file

model_latest_checkpoint_lstm = create_model()
# Load the previously saved weights
model_latest_checkpoint_lstm.load_weights(checkpoint_path)

In [ ]:
acc_lstm = model_lstm_1.evaluate(X_test1, Y_test)
print("Loss:", acc_lstm[0], " Accuracy:", acc_lstm[1])

pred = model_lstm_1.predict(X_test1)
y_pred = pred.argmax(axis=-1)
y_true = Y_test

In [ ]:
output_classes=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19']

def plot_confusion_matrix(y_true, y_pred, classifier_name):
    mtx = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(20,20))
    ax.set_title(classifier_name)
    sns.heatmap(mtx, xticklabels= output_classes, yticklabels=output_classes, cmap="Blues", annot=True, fmt='d', linewidths=.5,  cbar=False, ax=ax)
    #  square=True,
    plt.ylabel('true label')
    plt.xlabel('predicted label')

In [ ]:
plot_confusion_matrix(y_true, y_pred, classifier_name='DLSTM')
print(classification_report(y_true, y_pred,digits=4))
acc_lstm1 = accuracy_score(y_true, y_pred)
print(acc_lstm1)

DCNN+DLSTM

In [ ]:
img_rows = 40
img_cols = 50

X_train1 = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_val1 = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
X_test1 = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
print(X_train1.shape, X_val1.shape, X_test1.shape, Y_train.shape, Y_val.shape, Y_test.shape )

input_shape = (img_rows, img_cols, 1)

In [ ]:
activation='relu'

def create_model():
  cnnmodel = Sequential()   
  cnnmodel.add(Conv2D(128, kernel_size=(3, 3), activation=activation, input_shape=input_shape)) 
  cnnmodel.add(MaxPooling2D(2,2))
  cnnmodel.add(Conv2D(64, (3, 3), activation=activation))
  cnnmodel.add(MaxPooling2D(2,2))
  cnnmodel.add(Conv2D(64, kernel_size=(3, 3), activation=activation, input_shape=input_shape)) 
  cnnmodel.add(MaxPooling2D(2,2))
  cnnmodel.add(Flatten())
  
  model = tf.keras.models.Sequential()
  model.add(TimeDistributed(cnnmodel, ...))
  model.add(tf.compat.v1.keras.layers.CuDNNLSTM(100, input_shape=input_shape, return_sequences=True))
  model.add(tf.compat.v1.keras.layers.CuDNNLSTM(100, return_sequences=True))
  model.add(tf.compat.v1.keras.layers.CuDNNLSTM(100))
  model.add(tf.keras.layers.Dense(100, activation='relu'))
  model.add(tf.keras.layers.Dense(19, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model


In [ ]:
from keras.callbacks import ModelCheckpoint

model_ckpt= create_model()
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/PQ19/data/chp19.ckpt"
cp_callback = ModelCheckpoint(filepath=checkpoint_path,save_best_only=True, save_weights_only=True, verbose=1)

model_ckpt.summary()

In [ ]:
batchsize = 100
epochs = 1000

hist=model_ckpt.fit(X_train1, Y_train, batch_size=batchsize, epochs=epochs, validation_data=(X_val1, Y_val),callbacks=[cp_callback], verbose=1)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')  
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
#get the latest checkpoint file

model_latest_checkpoint_lstm = create_model()
# Load the previously saved weights
model_latest_checkpoint_lstm.load_weights(checkpoint_path)

In [ ]:
acc_dcnnlstm = model_latest_checkpoint_lstm.evaluate(X_test1, Y_test)
print("Loss:", acc_lstm[0], " Accuracy:", acc_lstm[1])

pred = model_latest_checkpoint_lstm.predict(X_test1)
y_pred = pred.argmax(axis=-1)
y_true = Y_test

In [ ]:
output_classes=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19']

def plot_confusion_matrix(y_true, y_pred, classifier_name):
    mtx = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(20,20))
    ax.set_title(classifier_name)
    sns.heatmap(mtx, xticklabels= output_classes, yticklabels=output_classes, cmap="Blues", annot=True, fmt='d', linewidths=.5,  cbar=False, ax=ax)
    #  square=True,
    plt.ylabel('true label')
    plt.xlabel('predicted label')

**WMV** weight updating

In [5]:
DCNNmostconfusedclasses=['C2','C10','C14','C16','C17','C18','C19']
DCNNbestpredictedclasses=['C1','C3','C4','C5','C6','C7','C8','C11','C12','C13']
DLSTMmostconfusedclasses=['C1','C7','C9']
DLSTMbestpredictedclasses=['C2','C10','C15','C16','C17','C18','C19']
DCNNLSTMmostconfusedclasses=['C4','C16','C17','C19']
DCNNLSTMbestpredictedclasses=['C1','C7','C9','C10','C15']

Wdcnn=1
Wdlstm=1
Wdcnnlstm = 1
alphai=0.7

predCNN = model_latest_checkpoint_cnn.predict(X_val1)
predCNN = pred.argmax(axis=-1)

predLSTM = model_latest_checkpoint_lstm.predict(X_val1)
predLSTM = pred.argmax(axis=-1)

predCNNLSTM = model_latest_checkpoint_cnnlstm.predict(X_val1)
predCNNLSTM = pred.argmax(axis=-1)

if predCNN in DCNNmostconfusedclasses :
  Wdcnn=Wdcnn+alphai
elif predCNN in DCNNbestpredictedclasses :
  Wdcnn=Wdcnn-alphai
else:
  Wdcnn=Wdcnn

if predLSTM in DLSTMmostconfusedclasses :
  Wdlstm=Wdlstm+alphai
elif predLSTM in DLSTMbestpredictedclasses :
  Wdlstm=Wdlstm-alphai
else:
  Wdlstm=Wdlstm

if predCNNLSTM in DCNNLSTMmostconfusedclasses :
  Wdcnn=Wdcnn+alphai
elif predCNNLSTM in DCNNLSTMbestpredictedclasses :
  Wdcnn=Wdcnn-alphai
else:
  Wdcnn=Wdcnn
